In [84]:
import sys
sys.path.append("../")
import gpt

In [85]:
file ="/home/luca/Documenti/sezione_copernicus/tmp/PM-MOO-MS-Copernicus_matteo.gpkg"

In [86]:
gp = gpt.read_file(file)
gp.list()

INFO: Failed to auto identify EPSG: 7
INFO: Failed to auto identify EPSG: 7
INFO: Failed to auto identify EPSG: 7
INFO: Failed to auto identify EPSG: 7


[' Contacts',
 ' Geological_Units',
 ' Linear_Features',
 ' Pyroclastic_Deposits',
 'layer_styles']

In [87]:
# we have loaded the data in memory.
# now let say we want to change " Geological_Units" to "geounits"
oldname = " Geological_Units"
newname = "geounits"
# we do the actual rename
gp.rename_layer(oldname, newname)

# we fix the table - using self so it can be plugged into the lib easily
def fix_style_layer_name_changed(self:gpt.Geopkg, oldname, newname):
    styles = self["layer_styles"]
    styles.loc[styles["f_table_name"] == oldname, "f_table_name"] = newname
    return styles # not really needed

    
styles = fix_style_layer_name_changed(gp, oldname, newname )

In [88]:
# now we want to change the name of a field:
old_field_name = "Type"
new_field_name = "geo_type"
layer_name = "geounits"

def ensure_just_one_default_style(style_table, layer_name):
    """perform some checks on the style table and clean if possible"""
    ids = list(styles.loc[styles["f_table_name"] == layer_name].index) # ids of the possible style for this layer
    defaults = list(styles.loc[ ids, "useAsDefault"]) # the default flag for these rows
    
    if len(ids) == 0:
        log.error("no style for this layer found")
        return
    
    if len(ids) == 1:
        id = ids[0] # use that as style
        
    else: # we select the first style that is also set as default
        try:
            first_default  = defaults.index(True)
            id = ids[first_default]
        except: # no style set as default
            log.warning("No styles with default flag. using as style the last inserted. may be a problem!")
            id = ids[-1] # chose the last inserted
    
    styles.loc[ids, "useAsDefault"] = False # clear default flags, just to be sure, we may actually want to delete those unused styles
    styles.loc[id, "useAsDefault"] = True # now set the default style
    

def rename_field(self:gpt.Geopkg, layer_name, old_field_name, new_field_name):
    # classical change
    layer = self[layer_name]
    layer.rename(columns={old_field_name:new_field_name}, inplace=True)
    
    # fix the style table
    styles = self["layer_styles"] # extract the style table - we should check whether it exists or not btw
    
    # these would not be needed actually, just an useful check
    ensure_just_one_default_style(styles, layer_name)
    
    # now we want to alter the qml content for all the styles connected with this layers
    ids = list(styles.loc[styles["f_table_name"] == layer_name].index)
    for id in ids:
        qml = styles.loc[id, "styleQML"]

        import xml.etree.ElementTree as ET
        tree = ET.fromstring(qml)
        renderer = tree.find("renderer-v2")
        
        if renderer.attrib["attr"] == old_field_name: # the field was used in this style
            renderer.attrib["attr"] = new_field_name
            newstyle = str(ET.tostring(tree, encoding='unicode', method='xml'))
            styles.loc[id,"styleQML"] = newstyle
        else: # nothing to do - not affected
            log.debug("the field were not used as filter for the style")
            return 

   
    

        # clear this because we cannot grant 1 to 1 with qml for now
        styles.loc[id, "styleSLD"] = ' '
        return 
    
    

rename_field(gp, layer_name, old_field_name, new_field_name)

In [89]:
gp.to_gpkg("altered.gpkg")